In [2]:
!pip install torch==1.4.0 torchvision==0.5.0
import torch
torch.cuda.get_device_name(0)

     |████████████████████████████████| 753.4MB 23kB/s 
     |████████████████████████████████| 4.0MB 50.7MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


'Tesla T4'

In [1]:
!nvidia-smi

Sat Jan 30 09:34:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    12W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Make dataset

In [ ]:
#@markdown Folder path with all the videos
inputPath = 'C:\\Users\\Heylon\\Documents\\cain\\caindatasetMinecraft\\input\\' #@param{type:"string"}
#@markdown Path to generate the Database
outputPath = 'C:\\Users\\Heylon\\Documents\\cain\\caindatasetMinecraft\\out\\' #@param{type:"string"}
#@markdown The bigger the value, the more alike the frames need to be to be considered a triplet.
psnr = 15 #@param{type:"number"}
#@markdown The original model was trained with images of res 448X256, this bool turn all frames to that resolution.
resize = True #@param{type:"boolean"}

#Frames to skip between triplets
step = 15

outputExtension = '.jpg'

#Original dataset information:
#Vimeo-90K triplets dataset contains 91701 triplets extracted from 15k video clips.
#Each triplet is a short RGB video sequence that consists of 3 frames with fixed resolution 448x256
import subprocess
import re
def getRes(inputPath):
    '''
    Gets the Resolution as a tuple from a given video path
    '''
    videoRes = [0, 0]
    result = subprocess.run(['ffmpeg', '-i', inputPath], stderr=subprocess.PIPE)
    lines = result.stderr.splitlines()

    print("------LINES------")
    for line in lines:
        decodedLine = line.decode('UTF-8')
        if 'Stream #0:0' in decodedLine:
            print(decodedLine)
            x = re.search(r"[,] ([0-9]+)[x]([0-9]+)", decodedLine)
            videoRes[0] = int(x.group(1))
            videoRes[1] = int(x.group(2))
            return tuple(videoRes)

def calculate_psnr(img1, img2, max_value=255):
    """"Calculating peak signal-to-noise ratio (PSNR) between two images."""
    i1 = PIL.Image.open(img1).convert('RGB')
    i2 = PIL.Image.open(img2).convert('RGB')

    mse = np.mean((np.array(i1, dtype=np.float32) - np.array(i2, dtype=np.float32)) ** 2)
    if mse == 0:
        return 100
    return 20 * np.log10(max_value / (np.sqrt(mse)))


def IsDiffScenes(img1, img2, diff=25):
  return calculate_psnr(img1, img2) <= diff


import os
import shutil
from shutil import copyfile
import time
import PIL
from PIL import Image
import numpy as np


if not os.path.isdir(outputPath):
  os.makedirs(outputPath)

if not os.path.isdir(outputPath + "sequences"):
  os.makedirs(outputPath + "sequences")

counter = 1
for video in sorted(os.listdir(inputPath)):
  temp = os.getcwd() + "/.temp/"
  path = inputPath + video
  if os.path.isdir(temp):
    shutil.rmtree(temp)
    #time.sleep(2)
  os.makedirs(temp)
  vidRes = None
  try:
    vidRes = getRes(path)
  except:
    pass

  if vidRes == None:
    continue

  scaleSize = 500
  rescale = ""
  if resize:
    if vidRes[0] < vidRes[1]:
      rescale="scale=" + str(scaleSize) +":-1,pad=ceil(iw/2)*2:ceil(ih/2)*2,"
    else:
      rescale="scale=-1:" + str(scaleSize) + ",pad=ceil(iw/2)*2:ceil(ih/2)*2,"
  !ffmpeg -hwaccel cuda -i "{path}" -vf "{rescale}mpdecimate" -vsync 0 -pix_fmt rgb24 -qscale:v 2 "{temp}%10d{outputExtension}"

  root =outputPath + "/sequences/" + str(counter).zfill(3) + "/"
  if not os.path.isdir(root):
    os.makedirs(root)


  triIndex = 1
  images = sorted(os.listdir(temp))
  for i in range(2, len(images),step) :
    im1 = temp + images[i-2]
    im2 =  temp + images[i-1]
    im3 =  temp + images[i]
    if IsDiffScenes(im1, im2, psnr) == False and IsDiffScenes(im2, im3, psnr) == False:
      print("Triplet Found")
      triPath = root + str(triIndex).zfill(5) + "/"
      if not os.path.isdir(triPath):
        os.makedirs(triPath)
      copyfile(im1, triPath + "im1" + outputExtension)
      copyfile(im2, triPath + "im2" + outputExtension)
      copyfile(im3, triPath + "im3" + outputExtension)
      triIndex +=1

  counter += 1

------LINES------
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, smpte170m/smpte170m/bt470m), 1920x1200 [SAR 1:1 DAR 8:5], 49791 kb/s, 59.95 fps, 59.94 tbr, 90k tbn, 120 tbc (default)


ffmpeg version N-100214-g95fd790c14 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3-win32 (GCC) 20200320
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-zlib --enable-libxml2 --enable-libfreetype --enable-libfribidi --enable-gmp --enable-lzma --enable-fontconfig --enable-opencl --enable-libvmaf --disable-vulkan --enable-libvorbis --enable-amf --enable-libaom --enable-avisynth --enable-libdav1d --enable-libdavs2 --enable-ffnvcodec --enable-cuda-llvm --disable-libglslang --enable-libass --enable-libbluray --enable-libmp3lame --enable-libopus --enable-libtheora --enable-libvpx --enable-libwebp --enable-libmfx --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librav1e --enable-librubberband --enable-schannel --enable

Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triple

ffmpeg version N-100214-g95fd790c14 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3-win32 (GCC) 20200320
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-zlib --enable-libxml2 --enable-libfreetype --enable-libfribidi --enable-gmp --enable-lzma --enable-fontconfig --enable-opencl --enable-libvmaf --disable-vulkan --enable-libvorbis --enable-amf --enable-libaom --enable-avisynth --enable-libdav1d --enable-libdavs2 --enable-ffnvcodec --enable-cuda-llvm --disable-libglslang --enable-libass --enable-libbluray --enable-libmp3lame --enable-libopus --enable-libtheora --enable-libvpx --enable-libwebp --enable-libmfx --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librav1e --enable-librubberband --enable-schannel --enable

Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triple

...

Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triple

ffmpeg version N-100214-g95fd790c14 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3-win32 (GCC) 20200320
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-zlib --enable-libxml2 --enable-libfreetype --enable-libfribidi --enable-gmp --enable-lzma --enable-fontconfig --enable-opencl --enable-libvmaf --disable-vulkan --enable-libvorbis --enable-amf --enable-libaom --enable-avisynth --enable-libdav1d --enable-libdavs2 --enable-ffnvcodec --enable-cuda-llvm --disable-libglslang --enable-libass --enable-libbluray --enable-libmp3lame --enable-libopus --enable-libtheora --enable-libvpx --enable-libwebp --enable-libmfx --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librav1e --enable-librubberband --enable-schannel --enable

Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triple

ffmpeg version N-100214-g95fd790c14 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3-win32 (GCC) 20200320
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-zlib --enable-libxml2 --enable-libfreetype --enable-libfribidi --enable-gmp --enable-lzma --enable-fontconfig --enable-opencl --enable-libvmaf --disable-vulkan --enable-libvorbis --enable-amf --enable-libaom --enable-avisynth --enable-libdav1d --enable-libdavs2 --enable-ffnvcodec --enable-cuda-llvm --disable-libglslang --enable-libass --enable-libbluray --enable-libmp3lame --enable-libopus --enable-libtheora --enable-libvpx --enable-libwebp --enable-libmfx --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librav1e --enable-librubberband --enable-schannel --enable

Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triplet Found
Triple

# Generate training and testing sets

In [ ]:
#@markdown The folder path of the generated Dataset
database = "C:\\Users\\Heylon\\Documents\\cain\\caindatasetMinecraft\\out\\" #@param{type:"string"}
#@markdown Percentage of frames that will be to the testing group.
testingPercentage = 0.2 #@param{type:"slider", min:0.1, max:0.9, step:0.1}

import random

if os.path.isfile(database + "tri_testlist.txt"):
  os.remove(database + "tri_testlist.txt")
if os.path.isfile(database + "tri_trainlist.txt"):
  os.remove(database + "tri_trainlist.txt")

from glob import glob
folders = glob(database + "sequences/*/")

paths = []

for i in range(0, len(folders)):
  triplets = glob(folders[i] + "/*/")
  for ii in range(0, len(triplets)):
    innerF = os.path.basename((os.path.dirname(triplets[ii])))
    outF = os.path.basename((os.path.dirname(folders[i])))
    paths.append(outF + "/" + innerF)

def partitionRankings(rawRatings, testPercent):
    howManyNumbers = int(round(testPercent*len(rawRatings)))
    shuffled = rawRatings[:]
    random.shuffle(shuffled)
    return shuffled[howManyNumbers:], shuffled[:howManyNumbers]

training, test = partitionRankings(paths, testingPercentage)


trainTxt = database + "/tri_trainlist.txt"
trainFile = open(trainTxt, 'w')

testTxt = database + "/tri_testlist.txt"
testFile = open(testTxt, 'w')

for txt in training:
  trainFile.write(txt + "\n")
trainFile.close()

for txt in test:
  testFile.write(txt + "\n")
testFile.close()

print(training)
print(test)


['019/00132', '001/00226', '003/00265', '003/00038', '020/00244', '002/00054', '008/00023', '023/00211', '010/00228', '005/00008', '018/00087', '002/00556', '020/00276', '010/00179', '002/00736', '011/00198', '012/00056', '018/00002', '004/00015', '016/00127', '011/00137', '011/00195', '004/00062', '008/00156', '016/00004', '002/00664', '021/00029', '017/00228', '014/00018', '020/00368', '010/00345', '002/00100', '010/00107', '021/00011', '019/00116', '005/00195', '008/00037', '002/00300', '014/00300', '017/00082', '010/00125', '020/00440', '022/00132', '005/00129', '017/00012', '023/00160', '019/00202', '023/00024', '002/00506', '008/00211', '024/00026', '004/00066', '002/00510', '008/00173', '007/00207', '006/00099', '005/00074', '010/00111', '007/00146', '002/00566', '002/00226', '001/00197', '023/00083', '023/00141', '017/00026', '011/00210', '017/00131', '007/00158', '010/00155', '002/00301', '001/00196', '008/00090', '011/00199', '020/00390', '020/00272', '022/00110', '006/00120'

# Train

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import shutil
shutil.copy('/content/drive/MyDrive/caintrain/out.7z','/content/model.7z')
!7z x /content/model.7z -aoa


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 1263678984 bytes (1206 MiB)

Extracting archive: /content/model.7z
--
Path = /content/model.7z
Type = 7z
Physical Size = 1263678984
Headers Size = 173621
Method = LZMA2:1536m
Solid = +
Blocks = 1

  0%      0% 5047 - out/sequences/020/00444tcmalloc: large alloc 1610612736 bytes == 0x5592c3b84000 @  0x7fb1a02bab6b 0x7fb1a02da379 0x5592c21c9be9 0x7fb19e956d9b 0x7fb19e952faf 0x7fb19ea80ec4 0x7fb19e96775e 0x7fb19e96d043 0x5592c21ee9d2 0x5592c2215f4f 0x5592c21c9200 0x7fb19f511bf7 0x5592c21c96da
                                     0% 5724           0% 5780 - out/sequences/001/00019/im3.j

In [ ]:
import os
os.chdir('/content/drive/MyDrive/caintrain/CAIN')
!CUDA_VISIBLE_DEVICES=0 python main.py --exp_name CAIN --batch_size 16 --test_batch_size 16 --dataset vimeo90k --model cain --loss 1*L1 --max_epoch 200 --lr 0.0002 --data_root /content/out

Namespace(batch_size=16, beta1=0.9, beta2=0.99, cuda=True, data_dir='data', data_root='/content/out', dataset='vimeo90k', depth=3, exp_name='CAIN', fix_loaded=False, img_fmt='png', log_dir='logs', log_iter=20, loss='1*L1', lpips=False, lr=0.0002, max_epoch=200, mode='train', model='cain', n_resblocks=12, num_frames=3, num_gpu=1, num_workers=5, random_seed=12345, resume=False, resume_exp=None, start_epoch=0, test_batch_size=16, test_mode='hard', up_mode='shuffle', use_tensorboard=False, val_batch_size=4, viz=False)
2021-01-30 09:44:07.155136: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Building model: CAIN
Preparing loss function:
1.000 * L1
# of parameters: 42780432
Train Epoch: 0 [0/285]	Loss: 0.087839	PSNR: 19.4634	Time(6.20)
Train Epoch: 0 [20/285]	Loss: 0.060979	PSNR: 25.9097	Time(16.02)
Train Epoch: 0 [40/285]	Loss: 0.033927	PSNR: 26.7823	Time(15.90)
Train Epoch: 0 [60/285]	Loss: 0.028436	PSNR: 26.8982	Time(